In [388]:
ktag="lda"

In [362]:
import numpy as np
import pandas as pd
import sys,os
import seaborn as sns
sns.set(style='whitegrid')
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelBinarizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score,roc_curve

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.datasets import make_regression

from sklearn.model_selection import train_test_split
from scipy import stats

from keras.optimizers import Adam

from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from imblearn.under_sampling import RandomUnderSampler,NearMiss
from imblearn.over_sampling import RandomOverSampler,SMOTE

from keras.models import Sequential,load_model
from keras.layers import LSTM
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from keras.layers import TimeDistributed,BatchNormalization
from keras.layers import Bidirectional

from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, History

from keras.utils import to_categorical
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # "0, 1" for multiple

In [90]:
stars=pd.read_pickle("/data/Hoaxy/star_networks_adoption_time_updated.pkl.gz")

In [240]:
stars.head()

,document_ids,from_user_id,to_user_id,norm_count
0,1099777,1025830696923922432,1,-0.065577
0,1099777,1025830696923922432,1,-0.065577
0,1099777,1025830696923922432,1,-0.065577
0,1099777,1025830696923922432,1,-0.065577
0,1099843,188760974,3,-0.027527


In [92]:
##stars["document_ids"].nunique(),stars["from_user_id"].nunique(),stars.shape[0]

In [93]:
assert (stars[["document_ids","from_user_id"]].drop_duplicates().shape[0]*4 == stars.shape[0])

In [235]:
stars=stars[["document_ids","from_user_id","to_user_id"]]

In [239]:
stars["norm_count"]=(stars["to_user_id"]-stars["to_user_id"].mean()) / stars["to_user_id"].std()

In [241]:
stars_qlist=stars.groupby(["document_ids","from_user_id"])["norm_count"].apply(list).reset_index(name="qstages")

In [242]:
stars_by_max_size=stars.groupby(["document_ids","from_user_id"])["norm_count"].max().reset_index(name="max_size")

In [243]:
stars_doc_by_max_size=stars_by_max_size.groupby("document_ids")["max_size"].max().reset_index(name="max_size")

In [244]:
##stars_doc_by_max_size=stars_doc_by_max_size[stars_doc_by_max_size["max_size"]>10]

In [245]:
stars_by_max_size_filtd=pd.merge(stars_by_max_size,stars_doc_by_max_size,on=["document_ids","max_size"],how="inner")

In [246]:
##stars_by_max_size_filtd.describe()

In [247]:
stars_qlist_filtd=pd.merge(stars_qlist,stars_by_max_size_filtd,on=["document_ids","from_user_id"],how="inner")

In [248]:
stars_qlist_filtd=stars_qlist_filtd[["document_ids","qstages"]]

In [249]:
stars_qlist_filtd.shape

(8764, 2)

## Language Models

In [389]:
pairs=pd.read_pickle("/data/Hoaxy/%s_doc_pairs.pkl.gz"%ktag)

In [390]:
pairs.shape

(1400000, 3)

In [391]:
## preparing data
example_pairs=pd.merge(pairs,stars_qlist_filtd,left_on=["document1"],right_on=["document_ids"],how="inner")

In [392]:
example_pairs=pd.merge(example_pairs,stars_qlist_filtd,left_on=["document2"],right_on=["document_ids"],how="inner")

In [393]:
example_pairs.shape

(548744, 7)

In [394]:
example_pairs.tail()

,document1,document2,class,document_ids_x,qstages_x,document_ids_y,qstages_y
548739,1099845,1100350,1,1099845,"[-0.0655768575938136, -0.0655768575938136, -0....",1100350,"[-0.0655768575938136, -0.0655768575938136, -0...."
548740,1099754,1100350,1,1099754,"[-0.008502160980427949, -0.008502160980427949,...",1100350,"[-0.0655768575938136, -0.0655768575938136, -0...."
548741,1099754,1100350,1,1099754,"[-0.008502160980427949, -0.008502160980427949,...",1100350,"[-0.0655768575938136, -0.0655768575938136, -0...."
548742,1099206,1099894,1,1099206,"[-0.027527059851556502, -0.027527059851556502,...",1099894,"[-0.0655768575938136, -0.046551958722685055, -..."
548743,1099206,1099894,1,1099206,"[-0.027527059851556502, -0.027527059851556502,...",1099894,"[-0.0655768575938136, -0.046551958722685055, -..."


In [395]:
example_pairs[example_pairs["class"]==0].shape

(237396, 7)

In [396]:
x1=example_pairs["qstages_x"].values
x2=example_pairs["qstages_y"].values

In [397]:
x1.shape

(548744,)

In [398]:
Y=example_pairs["class"].values

In [399]:
Y.shape

(548744,)

## RF

In [400]:
X=np.column_stack((x1,x2))

In [401]:
X=np.array([np.array(list(record)).flatten() for record in X])

In [402]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

In [403]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (411558, 8)
Training Labels Shape: (411558,)
Testing Features Shape: (137186, 8)
Testing Labels Shape: (137186,)


In [342]:
rf = RandomForestClassifier(n_estimators = 1000,max_depth=4,random_state = 42)
rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [343]:
Y_pred = rf.predict(X_test)

In [344]:
print(accuracy_score(Y_test, Y_pred))

# # Creating the Confusion Matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(prediction, Y_test)
# print(cm)

precision,recall,f1_score,support=precision_recall_fscore_support(Y_test, Y_pred)
scores="class\tprecision\trecall\tF1-score\tsupport\n"
for j in [0,1]:
    scores+="%s,%s,%s,%s,%s\n" % (str(j),str(precision[j]),str(recall[j]),str(f1_score[j]),str(support[j]))
print(scores)

0.6395050023702004
class	precision	recall	F1-score	support
0,0.6425281199785753,0.802157631954351,0.7135238019707754,22432
1,0.6324942033786022,0.43277239503654596,0.5139108494533221,17649



## LSTM

In [345]:
XL=np.column_stack((x1,x2))

In [346]:
XLM=np.array([np.array([record[0],record[1]]).T for record in XL])

In [347]:
##YL=to_categorical(Y,num_classes=2)

n_samples=XLM.shape[0]
n_timesteps=1
##n_features=2
YLM = Y.reshape(n_samples, n_timesteps)

In [348]:
XLM.shape,YLM.shape

((160324, 4, 2), (160324, 1))

In [349]:
XLM_train, XLM_test, YLM_train, YLM_test = train_test_split(XLM, YLM, test_size = 0.25, random_state = 42)

In [350]:
print('Training Features Shape:', XLM_train.shape)
print('Training Labels Shape:', YLM_train.shape)
print('Testing Features Shape:', XLM_test.shape)
print('Testing Labels Shape:', YLM_test.shape)

Training Features Shape: (120243, 4, 2)
Training Labels Shape: (120243, 1)
Testing Features Shape: (40081, 4, 2)
Testing Labels Shape: (40081, 1)


In [370]:
model = Sequential()

# # define CNN model
# model.add(TimeDistributed(Conv2D(1, (2,2), activation='relu', padding='same',input_shape=(None,4,2))))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Flatten()))

model.add(LSTM(8))
model.add(Dense(1, activation='sigmoid'))
opt=Adam(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [371]:
model.fit(XLM_train, YLM_train, validation_data=(XLM_test, YLM_test), epochs=4, batch_size=16,verbose=1)

Train on 120243 samples, validate on 40081 samples
Epoch 1/4
120243/120243 [==============================] - 80s 667us/step - loss: 0.6468 - acc: 0.6345 - val_loss: 0.6527 - val_acc: 0.6277
Epoch 2/4
120243/120243 [==============================] - 79s 654us/step - loss: 0.6435 - acc: 0.6403 - val_loss: 0.6393 - val_acc: 0.6438
Epoch 3/4
120243/120243 [==============================] - 79s 654us/step - loss: 0.6433 - acc: 0.6409 - val_loss: 0.6584 - val_acc: 0.6359
Epoch 4/4
120243/120243 [==============================] - 80s 668us/step - loss: 0.6456 - acc: 0.6373 - val_loss: 0.6545 - val_acc: 0.6232


In [353]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 8)                 352       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 9         
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________
None
